In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%cd gdrive/MyDrive/Colab\ Notebooks/4650Project/

/content/gdrive/MyDrive/Colab Notebooks/4650Project


Loading Data

In [3]:
sae = open("data/sae_samples.tsv").read().replace('\t', '')
sae = sae.split('\n')
sae = sae[0:-1]
print(len(sae))

FileNotFoundError: ignored

In [ ]:
sae[-2]

In [ ]:
aave = open("data/aave_samples.tsv").read().replace('\t', '')
aave = aave.split('\n')
print(len(aave))

In [ ]:
aave[-2]

Re-arrangement & Train-Test-Val Split

In [ ]:
data = {}
data['sae'] = sae
data['aave'] = aave

In [ ]:
import pandas as pd

In [ ]:
data = pd.DataFrame(data)
data

In [ ]:
data[0:1615].to_csv("data/training/train.csv", index=False)
data[1615:1817].to_csv("data/training/val.csv", index=False)
data[1817:].to_csv("data/training/test.csv", index=False)

#Training start here: 

In [ ]:
!pip install sentencepiece==0.1.91

In [ ]:
!pip install transformers

In [ ]:
import torch
import transformers
import pandas as pd
from pprint import pprint
from tqdm.notebook import tqdm

In [ ]:
train = pd.read_csv("data/training/train.csv")
train

In [ ]:
val = pd.read_csv("data/training/val.csv")
val

In [ ]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels, length):
        self.encodings = encodings
        self.labels = labels
        self.length = length

    def __getitem__(self, idx):
        encoding_item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        label_item = {key: torch.tensor(val[idx]) for key, val in self.labels.items()}
        return encoding_item, label_item

    def __len__(self):
        return self.length

def get_dataset(train, val):
    '''
    train: list of [sae, aave] for training 
    val: list of [sae, aave] for validation
    '''
    #from transformers import BartTokenizerFast
    from transformers import T5Tokenizer

    #tokenizer = BartTokenizerFast.from_pretrained('facebook/bart-base')
    tokenizer = T5Tokenizer.from_pretrained('t5-base')

    train_sae = tokenizer(list(train['sae']), padding=True)
    train_aave = tokenizer(list(train['aave']), padding=True)
    val_sae = tokenizer(list(val['sae']), padding=True)
    val_aave = tokenizer(list(val['aave']), padding=True)

    train_dataset = Dataset(train_sae, train_aave, length = len(list(train['sae'])))
    val_dataset = Dataset(val_sae, val_aave, length=len(list(val['sae'])))

    return train_dataset, val_dataset

In [ ]:
from transformers import T5Tokenizer
tokenizer = T5Tokenizer.from_pretrained('t5-base')
print(tokenizer)

In [ ]:
train_dataset, val_dataset = get_dataset(train, val)

In [ ]:
print(len(train_dataset))
print(type(train_dataset))

In [ ]:
import torch
from torch.nn import BCEWithLogitsLoss, BCELoss

from torch.utils.data import DataLoader

#from transformers import BartForConditionalGeneration, AdamW
from transformers import T5Tokenizer, T5ForConditionalGeneration, AdamW

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(torch.cuda.is_available())
print(device)

#model =  BartForConditionalGeneration.from_pretrained('facebook/bart-base')
model = T5ForConditionalGeneration.from_pretrained('t5-base')

model.to(device)
model.train()

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=False)

optim = AdamW(model.parameters(), lr=5e-5)

In [ ]:
WEIGHT_PATH = 'weights/T5'

In [ ]:
def train(epochs=10):
    train_loss_set = list()
    for epoch in tqdm(range(epochs)):
        for batch in train_loader:
            input, label = batch
            #print(input)

            optim.zero_grad()

            input_ids = input['input_ids'].to(device)
            attention_mask = input['attention_mask'].to(device)

            labels = label['input_ids'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs[0]
            train_loss_set.append(loss.item())

            # Backward pass
            loss.backward()

            optim.step()

        print("Finished epoch {}".format(epoch))

    model.save_pretrained(WEIGHT_PATH)
    return train_loss_set

In [ ]:
loss_values = train(10)

import matplotlib.pyplot as plt

plt.plot(loss_values)
plt.show()

In [ ]:
WEIGHT_PATH = 'weights/T5'

In [ ]:
DATAPATH = 'data/training/test.csv'
import torch

# from transformers import BartTokenizerFast
# tokenizer = BartTokenizerFast.from_pretrained('facebook/bart-base')

from transformers import T5Tokenizer
tokenizer = T5Tokenizer.from_pretrained('t5-base')

def get_data(path=DATAPATH):

  test = pd.read_csv("data/training/test.csv")
  # print(test)
  sae_list = list(test['sae'])
  aave_list = list(test['aave'])
  return sae_list, aave_list

class TestDataset(torch.utils.data.Dataset):
  def __init__(self, encodings, inputs, labels, length):
      self.encodings = encodings
      self.inputs = inputs
      self.labels = labels
      self.length = length

  def __getitem__(self, idx):
      encoding_item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
      return (encoding_item, self.inputs[idx], self.labels[idx])

  def __len__(self):
      return self.length

def get_dataset(sae_list, aave_list):

  sae_encodings = tokenizer(sae_list, padding=True)

  test_dataset = TestDataset(encodings = sae_encodings, inputs = sae_list, labels = aave_list, length = len(aave_list))

  return test_dataset

In [ ]:
sae_list, aave_list = get_data(path = 'data/training/test.csv')
test_dataset = get_dataset(sae_list, aave_list)
# print(test_dataset.inputs)
# print(test_dataset.labels)

In [ ]:
from transformers import T5ForConditionalGeneration
#from transformers import BartForConditionalGeneration

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm

def validate(val_dataset):
    model =  T5ForConditionalGeneration.from_pretrained(WEIGHT_PATH)
    #model =  BartForConditionalGeneration.from_pretrained(WEIGHT_PATH)
    model.to(device)
    model.eval()

    val_loader = DataLoader(val_dataset, batch_size=5, shuffle=False)

    print("Evaluating on {} questions".format(len(val_dataset)))
    
    org_inputs = []
    generated_texts = []
    references = []
    for batch in val_loader:
            input, org_input, reference = batch
            for o in org_input:
              org_inputs.append(o)
            for r in reference:
              references.append(r)

            input_ids = input['input_ids'].to(device)
            attention_mask = input['attention_mask'].to(device)

            with torch.no_grad():
                generated_sequences = model.generate(
                    input_ids=input_ids,
                    attention_mask = attention_mask,
                    max_length=600,
                    temperature=0.8,
                    top_k=5,
                    top_p=0.9,
                    repetition_penalty=0.8
                    )
                for sequence in generated_sequences:
                    hypothesis = tokenizer.decode(sequence, clean_up_tokenization_spaces=True)
                    hypothesis = hypothesis[5:]
                    #print(hypothesis)
                    end = hypothesis.index('</s>')
                    hypothesis = hypothesis[0:end]
                    generated_texts.append(hypothesis)
    return org_inputs, generated_texts, references

In [ ]:
org_inputs, generated_texts, references = validate(test_dataset)

In [ ]:
print(len(org_inputs))
print(len(generated_texts))
print(len(references))

In [ ]:
for i in range(0,10):
  print(org_inputs[i])
  print(references[i])
  print(generated_texts[i])
  print('==========================================================================')

In [ ]:
output = {}
output['org_inputs'] = org_inputs
output['references'] = references
output['generated_texts'] = generated_texts
output = pd.DataFrame(output)
output.to_csv("data/T5_output.csv", index=False)

In [ ]:
import torch
import nltk
import numpy as np
from datasets import load_metric
from datetime import datetime

def compute_metrics(predictions, labels):

    if type(labels) == list:
        labels = torch.tensor(labels)

    # Replace -100 in the labels as we can't decode them.
    labels = torch.where(labels != -100, labels, tokenizer.eos_token_id)

    predictions = predictions.cpu()
    labels = labels.cpu()

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    joined_preds = list(map(lambda x: ''.join(x).replace('\n', ''), decoded_preds))
    joined_labels = list(map(lambda x: ''.join(x).replace('\n', ''), decoded_labels))

    # Rouge expects a newline after each sentence
    nled_preds = list(map(lambda x: "\n".join(nltk.sent_tokenize(x)), joined_preds))
    nled_labels = list(map(lambda x: "\n".join(nltk.sent_tokenize(x)), joined_labels))

    rouge = load_metric("rouge")
    result = rouge.compute(predictions=nled_preds, references=nled_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [torch.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    # BERTScore # roberta-large => cuda out of memory
    bertscore = load_metric("bertscore")
    bs_result = bertscore.compute(predictions=joined_preds, references=joined_labels, lang='en')
    result = dict(result, **{'BS ' + k: np.mean(v) for k, v in bs_result.items() if k != 'hashcode'})

    # BLEU
    bleu = load_metric("bleu")
    preds = list(map(lambda x: list(map(lambda y: str(y), x)), joined_preds))
    refs = list(map(lambda x: [list(map(lambda y: str(y), x)),], joined_labels))
    result['bleu'] = bleu.compute(predictions=preds, references=refs)['bleu']

    return {k: v for k, v in result.items()}, joined_preds, joined_labels

In [ ]:
import transformers
transformers.logging.set_verbosity_error()

from tqdm import tqdm

metrics = {}
results = {k: [] for k in ['org_inputs', 'references', 'generated_texts']}
pred, eval = [], []
for i, x in enumerate(tqdm(val_data)):
    # if i == 10: break
    input_ids = torch.tensor(x['input_ids'][:prompt_lengths[i]]).cuda()
    prediction = model.generate(
        input_ids.unsqueeze(0),
        temperature=0.8,
        no_repeat_ngram_size=2,
        max_length=TOTAL_LEN,
        truncation=False,
    )[0]
    input_len = len(input_ids)
    results['org_inputs'] += [tokenizer.decode(input_ids).replace("\n", " ") + "\n",]
    
    pred += [prediction[input_len:].tolist(),]
    eval += [x['labels'][input_len:],]
    pred[-1] += [tokenizer.eos_token_id for _ in range(TOTAL_LEN - len(pred[-1]))]
    eval[-1] += [-100 for _ in range(TOTAL_LEN - len(eval[-1]))]

pred = torch.tensor(pred)
eval = torch.tensor(eval)

d, results['generated_texts'], results['references'] = compute_metrics(pred, eval)
for (k, v) in d.items():
    if k not in metrics.keys(): metrics[k] = []
    metrics[k] += [v,]